# import / 라이브러리 호출

In [18]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# Fixed RandomSeed / 랜덤시드 고정

In [19]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

# Data Load / 데이터 불러오기

In [20]:
train = pd.read_csv('open/train.csv')
test = pd.read_csv('open/test.csv')

In [21]:
train.head()

,ID,추석까지 남은 기간(주),쇼핑몰 구분,가격(원),프로모션 여부,도시 유형,지역 유형,쇼핑몰 유형,선물 유형,수요량
0,TRAIN_0000,1,쇼핑몰 15,212000,0,도시 6,지역 1,쇼핑몰 유형 2,명절혼합과일선물세트,28
1,TRAIN_0001,2,쇼핑몰 72,113000,0,도시 21,지역 1,쇼핑몰 유형 1,발효홍삼선물세트,27
2,TRAIN_0002,0,쇼핑몰 15,67000,0,도시 6,지역 1,쇼핑몰 유형 2,실속스팸선물세트,769
3,TRAIN_0003,1,쇼핑몰 13,206000,0,도시 12,지역 3,쇼핑몰 유형 1,자연산프리미엄버섯선물세트,27
4,TRAIN_0004,1,쇼핑몰 65,140000,0,도시 16,지역 2,쇼핑몰 유형 2,자연산새우선물세트,337


# Feature & Target Split / 독립변수, 종속변수로 나누기

In [22]:
train_x = train.drop(['ID', '수요량'], axis = 1)
train_y = train['수요량']

test_x = test.drop('ID', axis = 1)

In [23]:
train_x.head()

,추석까지 남은 기간(주),쇼핑몰 구분,가격(원),프로모션 여부,도시 유형,지역 유형,쇼핑몰 유형,선물 유형
0,1,쇼핑몰 15,212000,0,도시 6,지역 1,쇼핑몰 유형 2,명절혼합과일선물세트
1,2,쇼핑몰 72,113000,0,도시 21,지역 1,쇼핑몰 유형 1,발효홍삼선물세트
2,0,쇼핑몰 15,67000,0,도시 6,지역 1,쇼핑몰 유형 2,실속스팸선물세트
3,1,쇼핑몰 13,206000,0,도시 12,지역 3,쇼핑몰 유형 1,자연산프리미엄버섯선물세트
4,1,쇼핑몰 65,140000,0,도시 16,지역 2,쇼핑몰 유형 2,자연산새우선물세트


# Data Preprocessing / 데이터 전처리

In [24]:
ordinal_features = ['쇼핑몰 구분', '도시 유형', '지역 유형', '쇼핑몰 유형', '선물 유형']

for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(train_x[feature])
    train_x[feature] = le.transform(train_x[feature])

    # train데이터에서 존재하지 않았던 값이 test 데이터에 존재할 수도 있습니다.
    # 따라서 test 데이터를 바로 변형시키지 않고 고유값을 확인후 test 데이터를 변환합니다.
    # Data Leakage를 발생시키지 않기 위함이니, 반드시 주의해주세요.
    for label in np.unique(test_x[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test_x[feature] = le.transform(test_x[feature])

In [25]:
train_x.head()

,추석까지 남은 기간(주),쇼핑몰 구분,가격(원),프로모션 여부,도시 유형,지역 유형,쇼핑몰 유형,선물 유형
0,1,6,212000,0,20,0,1,9
1,2,69,113000,0,13,0,0,15
2,0,6,67000,0,20,0,1,22
3,1,4,206000,0,3,2,0,32
4,1,61,140000,0,7,1,1,30


# Regression Model Definition / 회귀 모델 정의

In [26]:
regressor = xgb.XGBRegressor(learning_rate=0.1, max_depth=4, min_child_weight=2, n_estimators= 300, random_state = 1004)

# Model fitting / 모델 학습

In [28]:
regressor.fit(train_x, train_y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=4,
             min_child_weight=2, missing=nan, monotone_constraints='()',
             n_estimators=300, n_jobs=64, num_parallel_tree=1,
             random_state=1004, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

# Inference / 추론

In [29]:
preds = regressor.predict(test_x)

# submit / 제출

In [32]:
submission = pd.read_csv('open/sample_submission.csv')

In [33]:
submission['수요량'] = preds
submission.head()

,ID,수요량
0,TEST_0000,218.083298
1,TEST_0001,-4.601458
2,TEST_0002,275.048523
3,TEST_0003,209.125046
4,TEST_0004,275.104156


In [34]:
submission.to_csv('./xgb_ver1.csv', index = False)